# CS 1656 – Introduction to Data Science 

## Instructor: Alexandros Labrinidis / Teaching Assistant: Evangelos Karageorgos
### Additional credits: Xiaoting Li, Tahereh Arabghalizi, Zuha Agha, Anatoli Shein, Phuong Pham
## Recitation : SQL via Data API
---
In this recitation, you will execute SQL queries on real data by connecting to the open data portal of [Western Pennsylavnia Regional Data Center](https://www.wprdc.org/) and requesting data via API calls.  

In [ ]:
import json
from datetime import datetime, timedelta, date
import requests
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

We will be using Allegheny County Restaurant/Food Facility Inspection Violation Dataset found here https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations. This dataset contains violation data from actual routine inspections by one of health department staff's members for the last two years. It should be fun to find out inspection results for places where we eat in Pittsburgh! =)

In [ ]:
wprdc_api_endpoint = "https://data.wprdc.org/api/3/action/datastore_search_sql"

# id for database table
resource_id = "1a1329e2-418c-4bd3-af2c-cc334e7559af"

# Get the date from 270 days ago)
# end_date = datetime.now()
# start_date = end_date - timedelta(days=270)

# Get two date endpoints
start_date = date(2018, 9, 1)
end_date = date(2019, 6, 1)

# Convert to a string the format the the data center accepts (yyyy-mm-dd)
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")

# SQL query we'll use in API call to request data
query = """
SELECT *
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}'""".format(resource_id, start_str, end_str, "Pittsburgh")

# Make WPRDC API Call
response = requests.get(wprdc_api_endpoint, {'sql': query})

# Parse response JSON into python dictionary
response_data = json.loads(response.text)

# Convert dictionary to dataframe
df = pd.DataFrame.from_dict(response_data['result']['records'])

# Print the number of rows
print(df.shape[0], "rows total")
print(df.columns)
df.head()


Details of dataset attributes are below. ((Taken from https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations/resource/1a1329e2-418c-4bd3-af2c-cc334e7559af)
![title](attribs.png)

## Queries

__Q1) Find all unique decription categories of violation in Pittsburgh restaurants over the time span (violation description[violation]).__

In [ ]:
query = """
SELECT DISTINCT "description_new" as violation
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' """.format(resource_id, start_str, end_str, "Pittsburgh")

response = requests.get(wprdc_api_endpoint, {'sql': query})

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

__Q2) Find restaurants in Pittsburgh with no violations in at least one decription category (facility name[facility], number of violations[count]).__

In [ ]:
query = """
SELECT "facility_name" as facility, COUNT("description_new") as count
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' AND "rating" <> '{}'
GROUP BY "facility_name" """.format(resource_id, start_str, end_str, "Pittsburgh", "V")

response = requests.get(wprdc_api_endpoint, {'sql': query})

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

### Tasks

__Tasks 1 to 4 must be implemented in Task.py__

__Task 5 must be implemented in the notebook__

__T1) Find top 30 restaurants in Pittsburgh with maximum number of violations (facility name[facility], number of violations[count]).__

Lets look more closely into the inspection results of Pitt facilities.

__T2) Find the category descriptions and their high, medium, low risk ratings for all violations at facilities that start with 'Pitt' over the past nine months (facility name[facility], violation description[violation], rating[rating], high[high], medium[medium], low[low]).__

Now lets look at all facilities that contain word 'Pitt'.

__T3) Find the category descriptions and their high, medium, low risk ratings for all violations at all facilities that have word 'Pitt' in their name. Note that results that contain word 'Pitt' as part of another word (e.g. 'Pittsburgh') should not be included (facility name[facility], violation description[violation], rating[rating], high[high], medium[medium], low[low])__

__T4) Find top 20 facilities that have word 'Pitt' in their name and have the highest counts of violations (facility name[facility], number of violations[count]).__

Let's make a bar graph of these counts now

__T5) Create a bar graph of the counts from T4).__